In [43]:
#######################################################
# 2023/10/4    11:00 

#######################################################

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.init as init
from torch.autograd import Variable
import random 
import numpy as np 
import torch.nn.functional as F
from sklearn.utils import resample


# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


class MLPDiffusion(nn.Module):    
    def __init__(self, d, n_steps):
        super(MLPDiffusion,self).__init__()
        num_units = d

        self.layer1 = nn.Linear(d, num_units, bias=True)
        self.layer2 = nn.Linear(num_units, num_units, bias=True)
        # self.layer3 = nn.Linear(num_units, num_units)
        self.layer4 = nn.Linear(num_units, d, bias=True)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()                 # self.tanh = nn.Tanh()  relu更容易收敛
        # self.bn_layers = nn.ModuleList([nn.BatchNorm1d(num_units) for _ in range(3)])
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(num_units) for _ in range(2)])

        # self.step_embeddings = nn.ModuleList([nn.Embedding(n_steps,num_units) for _ in range(3)])
        self.step_embeddings = nn.ModuleList([nn.Embedding(n_steps,num_units) for _ in range(2)])

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                if m == self.layer4:
                    # Xavier initialization for the layer with sigmoid activation
                    init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        init.constant_(m.bias, 0)
                else:
                    # He initialization for layers with ReLU activation
                    init.kaiming_uniform_(m.weight, nonlinearity='relu')
                    if m.bias is not None:
                        init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

    def forward(self, x, t):
        for idx, (embedding_layer, bn_layer) in enumerate(zip(self.step_embeddings, self.bn_layers)):
            t_embedding = embedding_layer(t)
            # x = self.layer1(x) if idx == 0 else self.layer2(x) if idx == 1 else self.layer3(x)
            x = self.layer1(x) if idx == 0 else self.layer2(x)
            x += t_embedding
            x = bn_layer(x)
            x = self.relu(x)
        
        x = self.layer4(x)
        x = self.sigmoid(x)
        return x


class MLPDiffusionWithLambda(nn.Module):
    def __init__(self, d, n_steps):
        super(MLPDiffusionWithLambda, self).__init__()
        self.diffNet = MLPDiffusion(d, n_steps)
        # 初始化lambda_weight 为一个较小的正值，例如0.5, 并使其为可学习的参数
        self.lambda_weight = nn.Parameter(torch.tensor(0.5))  # 如果是6的话得写成小数形式：6.  
    
    def forward(self, x, t):
        return self.diffNet(x, t)


class Diffusion(object):  # 注意：这里的batchsize和GAN里面的顺序不一样
    def __init__(self, dim, lr, epoches, batchsize=32):
        # 1. 实例化的参数
        self.dim = dim 
        self.batchsize = batchsize 
        self.lr = lr 
        self.epoches = epoches 


        # 2. 设置一些参数
        self.num_steps = 100    # 即T,对于步骤，一开始可以由beta, 分布的均值和标准差来共同确定
        self.betas = torch.linspace(-6, 6, self.num_steps)#制定每一步的beta, size:100
        self.betas = torch.sigmoid(self.betas)*(0.5e-2 - 1e-5)+1e-5   
        # beta是递增的，最小值为0.00001,最大值为0.005, sigmooid func
        # 像学习率一样的一个东西，而且是一个比较小的值，所以就有理由假设逆扩散过程也是一个高斯分布
        #计算alpha、alpha_prod、alpha_prod_previous、alpha_bar_sqrt等变量的值
        self.alphas = 1 - self.betas    # size: 100
        self.alphas_prod = torch.cumprod(self.alphas, 0)    # size: 100
        # 就是让每一个都错一下位
        self.alphas_prod_p = torch.cat([torch.tensor([1]).float(), self.alphas_prod[:-1]],0)  # p表示previous  
        # alphas_prod[:-1] 表示取出 从0开始到倒数第二个值
        self.alphas_bar_sqrt = torch.sqrt(self.alphas_prod)
        self.one_minus_alphas_bar_log = torch.log(1 - self.alphas_prod)
        self.one_minus_alphas_bar_sqrt = torch.sqrt(1 - self.alphas_prod)

        assert self.alphas.shape == self.alphas_prod.shape == self.alphas_prod_p.shape ==\
        self.alphas_bar_sqrt.shape == self.one_minus_alphas_bar_log.shape\
        == self.one_minus_alphas_bar_sqrt.shape


        # 3.初始化去噪模型
        self.Denoise = MLPDiffusionWithLambda(self.dim, self.num_steps)

        # 4.损失函数
        self.loss = nn.MSELoss()

        # 5.优化器
        # weight_decay=1e-5   添加L2正则化，权重衰减     self.lr = 0.005
        self.optimizer = optim.Adam(self.Denoise.parameters(), lr=self.lr, weight_decay=1e-5)

    #前向加噪过程，计算任意时刻加噪后的xt，基于x_0和重参数化
    def q_x(self, x_0, t, center, cov):
        """可以基于x[0]得到任意时刻t的x[t]"""
        
        noise = np.random.multivariate_normal(center, cov, x_0.shape[0])  
        noise = torch.from_numpy(np.maximum(np.minimum(noise, np.ones(( x_0.shape[0], self.dim))),
                                             np.zeros(( x_0.shape[0], self.dim)))).float()


        # noise = torch.randn_like(x_0)   # noise是从某分布中生成的随机噪声
        alphas_t = self.alphas_bar_sqrt[t]
        alphas_1_m_t = self.one_minus_alphas_bar_sqrt[t]

        xt = alphas_t * x_0 + alphas_1_m_t * noise
        return xt #在x[0]的基础上添加噪声
        # 上面就可以通过x0和t来采样出xt的值

    def diffusion_loss_fn(self, x_0, negative_samples, center, cov):
        ''' 
        x_0: positive_samples
        '''
        # 使用ReLU确保lambda_weight始终为正
        lambda_value = torch.relu(self.Denoise.lambda_weight)

        # 为了确保正样本的损失（loss_positive）在整体损失中占有更大的权重
        # 模型的目标是更加关注正样本，并尽可能让生成的样本远离负样本
        # 使用clamp确保lambda_weight在[a, b]范围内  
        lambda_weight = torch.clamp(lambda_value, min=0.5, max=0.1)

        # n_steps为中的时间步数，这里是500步
        batch_size = x_0.shape[0]
        n_steps = self.num_steps

        x_0 = torch.from_numpy(x_0).float()
        negative_samples = torch.from_numpy(negative_samples).float()

        t = torch.full((batch_size,), n_steps-1)
        t = t.unsqueeze(-1)

        xt = self.q_x(x_0, t, center, cov)

        output = self.Denoise(xt, t.squeeze(-1))
        # epsilon = (xt - x_0 * self.alphas_bar_sqrt[t]) / self.one_minus_alphas_bar_sqrt[t]  
        # 根据公式反推epsilon, 以便进行损失计算
        # return (epsilon - output).square().mean()

        # 正样本的重建误差
        loss_positive = (x_0 - output).square().mean()

        # 负样本的L2距离
        differences = output.unsqueeze(1) - negative_samples
        loss_negative = -torch.mean((differences ** 2).sum(dim=2))

        # 负样本的余弦距离
        # cosine_dists = 1.0 - F.cosine_similarity(output.unsqueeze(1), negative_samples, dim=2)
        # loss_negative = torch.mean(cosine_dists)

        # loss_negative = -torch.mean(torch.norm(output.unsqueeze(1) - negative_samples, dim=2)**2)

        # loss3: 计算两两之间的距离
        distances = torch.cdist(output, output)
        # 将对角线上的值(每个解与自身的距离)设置为0
        mask = torch.eye(len(output)) == 1
        distances.masked_fill(mask, 0)
        #计算每个解的多样性度量，即与其他解的平均距离
        diversity_measures = distances.sum(1) / (len(output) - 1)
        # 计算整体的多样性度量
        overall_diversity = diversity_measures.mean()
        # alpha是一个超参数，表示多样性正则化的权重
        alpha = -0.1  # 注意这里的 alpha为负，是为了在total_loss减去多样性度量，因为我们希望鼓励更大的多样性

        # 整合损失 = positive + negative  + diversity
        total_loss = loss_positive + lambda_weight * loss_negative + alpha * overall_diversity

        # return (x_0 - output).square().mean()
        return total_loss

    def train(self, positive_samples, negative_samples):
        # def train(self, pop_dec, samples_pool):
        ''' 
        pop_dec: shape(32, 31)   
        samples_pool.shape=(10, 31)是当前种群中表现最好的10个解，计算他们的均值和方差，用以生成随机噪声，即作为随机噪声的均值和方差
        
        positive_sample 用于训练正样本，并且作为center, cov的标尺, 这里只选前10个样本用于训练

        negative_sample 用于训练的负样本，余下的90个样本
        '''
        self.Denoise.train()
        n, d = np.shape(positive_samples)
        indices = np.arange(n)  # indices=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,..., 30, 31])
        
        center = np.mean(positive_samples[:10, :], axis=0)  # (31,1)  axis=0，对第一个维度求均值    下面的 cov 矩阵提供了一个关于这10个样本在31个特征上相互关系的全面视图。
        cov = np.cov(positive_samples[:10, :].reshape((d, positive_samples[:10, :].size // d)))#  (10, 31)->(31, 10)  conv=(31,31)  np.cov 函数用于计算协方差矩阵   samples_pool.shape=(10, 31),   
        
        # 定义目标样本数量
        target_samples = 50

        # 过采样正样本至50个
        upsampled_positive_samples = resample(positive_samples, 
                                            replace=True, 
                                            n_samples=target_samples,
                                            random_state=123)

        # 欠采样负样本至50个
        downsampled_negative_samples = resample(negative_samples, 
                                        replace=False, 
                                        n_samples=target_samples,
                                        random_state=123)




        for epoch in range(self.epoches):
            loss = 0
            self.optimizer.zero_grad()
            loss = self.diffusion_loss_fn(upsampled_positive_samples, downsampled_negative_samples, center, cov)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.Denoise.parameters(), 1.)
            self.optimizer.step()
            print("Epoch[{}], loss: {:.5f}".format(epoch, loss))

        random.shuffle(indices)
        positive_samples = positive_samples[indices, :]   # 感觉这里应该加上label = labels[indices, :]


    def p_sample_loop(self, x_T):
        # """从x[T]恢复x[T-1]、x[T-2]|...x[0]"""
        cur_x = x_T

        x_0 = self.p_sample(cur_x, self.num_steps - 1)
        return x_0

    def p_sample(self, x, t): # 参数重整化的过程
        """从x[t]采样t-1时刻的重构值，即从x[t]采样出x[t-1]"""
        t = torch.tensor([t])
        x_0 = self.Denoise(x,t)

        return x_0 
    

    def generate(self, sample_noises, population_size):# population_size=100
        self.Denoise.eval()
        center = np.mean(sample_noises, axis=0).T
        cov = np.cov(sample_noises.T)

        noises = np.random.multivariate_normal(center, cov, population_size)
        noises = torch.from_numpy(np.maximum(np.minimum(noises, np.ones((population_size, self.dim))),
                                             np.zeros((population_size, self.dim)))).float()

        with torch.no_grad():
            # decs= self.p_sample_loop(Variable(noises.cpu()).float(), center, cov).cpu().data.numpy()
            decs= self.p_sample_loop(Variable(noises.cpu()).float()).cpu().data.numpy()
    
        return decs 

In [ ]:
import torch

# 假设你有一个批次的生成解，维度为(batch_size, n_features)
generated_solutions = torch.randn((batch_size, n_features))

# 计算两两之间的距离
distances = torch.cdist(generated_solutions, generated_solutions)

# 将对角线上的值（每个解与自身的距离）设置为0
mask = torch.eye(batch_size) == 1
distances.masked_fill_(mask, 0)

# 计算每个解的多样性度量，即与其他解的平均距离
diversity_measures = distances.sum(1) / (batch_size - 1)

# 计算整体的多样性度量
overall_diversity = diversity_measures.mean()

# 假设你的主要损失为loss_main，然后将多样性正则化纳入总损失中
alpha = 0.1  # 这是一个超参数，表示多样性正则化的权重
total_loss = loss_main - alpha * overall_diversity  # 注意，这里是减去多样性度量，因为我们希望鼓励更大的多样性

# 接下来，你可以使用该损失来进行模型的更新


In [20]:
import torch 
a = torch.randn(2,3)
print(len(a))

2


In [44]:
import random 
import numpy as np 
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

# 生成一个形状为(100, 31)的随机数组
random_array = np.random.rand(100, 31)

# 沿着数组的第二个轴（axis=1）计算每行的和
row_sums = np.sum(random_array, axis=1)

# 使用NumPy的广播（broadcasting）机制进行规范化
normalized_array = random_array / row_sums[:, np.newaxis]

# 指定positive_samples的索引
positive_indices = [0, 2, 5, 9, 10, 11, 30, 22, 32, 12]

# 创建包含所有索引的列表
all_indices = list(range(100))

# 从all_indices中移除positive_indices 
negative_indices = list(set(all_indices) - set(positive_indices))

# 提取positive_samples 和 negative_samples 
positive_samples = normalized_array[positive_indices, :]
negative_samples = normalized_array[negative_indices, :]



# 假设你已经生成了positive_samples
# 为了使用SMOTE，我们还需要标签
# y_positive = [1] * len(positive_samples)

# # 临时添加一个负样本和其对应的标签
# X_temp = np.vstack([positive_samples, np.zeros((1, positive_samples.shape[1]))])
# y_temp = y_positive + [0]

# # 计算过采样比例
# oversampling_ratio = (3 * len(positive_samples) - len(positive_samples)) / len(positive_samples)

# # 使用SMOTE
# smote = SMOTE(sampling_strategy=oversampling_ratio,  
#               random_state=42, 
#               k_neighbors=5)

# X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

# 现在，删除临时的负样本并只选择正样本
# X_resampled = X_resampled[y_resampled == 1]
# y_resampled = y_resampled[y_resampled == 1]




# 你还需要对应的标签
y_positive = [1] * len(positive_samples)
y_negative = [0] * len(negative_samples[:50, :])
# negative_samples = normalized_array[negative_indices, :]

# 拼接数据
X = np.vstack((positive_samples, negative_samples[:50, :]))
y = np.array(y_positive + y_negative)

# 使用SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5)  # 设置k_neighbors为一个合适的值
X_resampled, y_resampled = smote.fit_resample(X, y)

# 现在，删除临时的负样本并只选择正样本
X_resampled = X_resampled[y_resampled == 1]
y_resampled = y_resampled[y_resampled == 1]

# X_resampled和y_resampled现在包含了原始数据和合成的正样本
# 你可以选择从X_resampled中分离出增强的正样本，如果需要的话

In [45]:
print(X_resampled.shape)
print(y_resampled)

(50, 31)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
# 定义目标样本数量
target_samples = 50

# 过采样正样本至50个
upsampled_positive_samples = resample(positive_samples, 
                                    replace=True, 
                                    n_samples=target_samples,
                                    random_state=123)

# 欠采样负样本至50个
downsampled_negative_samples = resample(negative_samples, 
                                replace=False, 
                                n_samples=target_samples,
                                random_state=123)


In [27]:
print(type(upsampled_positive_samples))
print(upsampled_positive_samples.shape)
unique_rows = np.unique(upsampled_positive_samples, axis=0)
num_unique_samples = unique_rows.shape[0]
print(num_unique_samples)

<class 'numpy.ndarray'>
(50, 31)
13


In [46]:
net = Diffusion(31, 0.001, 50)


In [47]:
# net.train(positive_samples, negative_samples)
net.train(X_resampled, negative_samples)


Epoch[0], loss: -0.58181
Epoch[1], loss: -0.59182
Epoch[2], loss: -0.60161
Epoch[3], loss: -0.60466
Epoch[4], loss: -0.60684
Epoch[5], loss: -0.62004
Epoch[6], loss: -0.62330
Epoch[7], loss: -0.63559
Epoch[8], loss: -0.63724
Epoch[9], loss: -0.64483
Epoch[10], loss: -0.65134
Epoch[11], loss: -0.65943
Epoch[12], loss: -0.66600
Epoch[13], loss: -0.67557
Epoch[14], loss: -0.68218
Epoch[15], loss: -0.69517
Epoch[16], loss: -0.69803
Epoch[17], loss: -0.69756
Epoch[18], loss: -0.71543
Epoch[19], loss: -0.71731
Epoch[20], loss: -0.72729
Epoch[21], loss: -0.73643
Epoch[22], loss: -0.74311
Epoch[23], loss: -0.75065
Epoch[24], loss: -0.75600
Epoch[25], loss: -0.76384
Epoch[26], loss: -0.77162
Epoch[27], loss: -0.77763
Epoch[28], loss: -0.78343
Epoch[29], loss: -0.79851
Epoch[30], loss: -0.80150
Epoch[31], loss: -0.80766
Epoch[32], loss: -0.81598
Epoch[33], loss: -0.82572
Epoch[34], loss: -0.83238
Epoch[35], loss: -0.83898
Epoch[36], loss: -0.84747
Epoch[37], loss: -0.85606
Epoch[38], loss: -0.86